### 2.8 표시자/더미 변수 계산하기

In [1]:
import pandas as pd

In [2]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'], 'data1': range(6)})
df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [3]:
df['key']

0    b
1    b
2    a
3    c
4    a
5    b
Name: key, dtype: object

#### - get_dummies(key) : 더미의 가변수를 만들어주는 함수, 결측값을 제외하고 0과 1로 구성된 더미값이 만들어짐
* prefix : 더미 변수의 컬럼명 앞에 붙을 이름 지정하는 옵션

In [4]:
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [7]:
dummies = pd.get_dummies(df['key'], prefix='key')
dummies

,key_a,key_b,key_c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [6]:
df[['data1']]

,data1
0,0
1,1
2,2
3,3
4,4
5,5


In [9]:
df_with_dummy = df[['data1']].join(dummies)
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


## 데이터 준비하기 : 조인, 병합, 변형

### 1. 계층적 색인

In [10]:
import numpy as np

In [12]:
data = pd.Series(np.random.randn(9),
                 index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1    0.537183
   2   -1.119420
   3   -1.524380
b  1    1.325549
   3    0.578407
c  1   -0.103242
   2   -0.338541
d  2   -1.501255
   3    1.806860
dtype: float64

#### - data.index 해보면 MultiIndex라고 나옴

In [13]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [14]:
data['b']

1    1.325549
3    0.578407
dtype: float64

In [15]:
data['b':'c']

b  1    1.325549
   3    0.578407
c  1   -0.103242
   2   -0.338541
dtype: float64

In [16]:
data.loc[:, 2]

a   -1.119420
c   -0.338541
d   -1.501255
dtype: float64

In [17]:
data

a  1    0.537183
   2   -1.119420
   3   -1.524380
b  1    1.325549
   3    0.578407
c  1   -0.103242
   2   -0.338541
d  2   -1.501255
   3    1.806860
dtype: float64

In [18]:
data.unstack()

,1,2,3
a,0.537183,-1.119420,-1.524380
b,1.325549,NaN,0.578407
c,-0.103242,-0.338541,NaN
d,NaN,-1.501255,1.806860


In [20]:
data.unstack().stack()

a  1    0.537183
   2   -1.119420
   3   -1.524380
b  1    1.325549
   3    0.578407
c  1   -0.103242
   2   -0.338541
d  2   -1.501255
   3    1.806860
dtype: float64

In [21]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                     columns=[['Ohio', 'Ohio', 'Colorado'],
                              ['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

#### - 인덱스, 컬럼이 두개이므로 이름 지정도 두개 해줘야함

In [23]:
frame.index.names = ['key1', 'key2']

In [24]:
frame

Ohio     Colorado
          Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [25]:
frame.columns.names = ['state', 'color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [27]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

### 1.1 계층의 순서를 바꾸고 정렬하기

In [28]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [29]:
frame.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

#### - key1이 level:0, key2가 level:1

In [31]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

### - 1.2 계층별 요약 통계

In [32]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [33]:
frame.sum(level=1)
#frame.sum(level='key2') 와 같음

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [34]:
frame = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1),
                      'c': ['one', 'one', 'one', 'two', 'two',
                            'two', 'two'],
                      'd': [0, 1, 2, 0, 1, 2, 3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


#### - set_index에 여러 개 넣으면 다중 인덱스 됨
* drop=False : 원래있던 자리에 데이터 그대로 두고 인덱스로도 가져옴

In [36]:
frame2 = frame.set_index(['c', 'd'])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [37]:
frame.set_index(['c', 'd'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [38]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


### 2. 데이터 합치기

### 2.1 데이터베이스 스타일로 DataFrame 합치기

In [39]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df2 = pd.DataFrame({'key': ['a', 'b', 'd'],
                    'data2': range(3)})

In [40]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


#### - pd.merge(a, b) : 두 곳에 다 있는 데이터만 병합해줌, default가 inner 

In [41]:
pd.merge(df1, df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [42]:
pd.merge(df1, df2, on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [43]:
df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'], 'data1': range(7)})
df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'], 'data2': range(3)})

In [44]:
df3

,lkey,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [45]:
df4

,rkey,data2
0,a,0
1,b,1
2,d,2


#### - df3에 대해서는 lkey로, df4에 대해서는 rkey를 키로 사용

In [46]:
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


In [47]:
pd.merge(df1, df2, how='outer')

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


### 2.2 색인 병합하기

In [48]:
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'], 'value': range(6)})
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])

In [49]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [50]:
right1

,group_val
a,3.5
b,7.0


#### - right_index=True: 인덱스를 사용해서 merge하기 

In [51]:
pd.merge(left1, right1, left_on='key', right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [52]:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [53]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                     index=['a', 'c', 'e'],
                     columns=['Ohio', 'Nevada'])
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                      index=['b', 'c', 'd', 'e'],
                      columns=['Missouri', 'Alabama'])

In [54]:
left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [55]:
right2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [56]:
pd.merge(left2, right2, left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
c,3.0,4.0,9.0,10.0
e,5.0,6.0,13.0,14.0


#### - a.join(b) : 인덱스를 가지고 병합함, default가 left기준이므로 a의 데이터는 반드시 들어감

In [57]:
left2.join(right2)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
c,3.0,4.0,9.0,10.0
e,5.0,6.0,13.0,14.0


In [58]:
left2.join(right2, how='inner')

,Ohio,Nevada,Missouri,Alabama
c,3.0,4.0,9.0,10.0
e,5.0,6.0,13.0,14.0


In [59]:
left2.join(right2, how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


#### - 인덱스 달라도 join 사용할 수 있음

In [60]:
left1.join(right1, on='key')

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


### 2.3 축 따라 이어붙이기

In [61]:
arr = np.arange(12).reshape((3, 4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

#### - numpy 사용해서 행으로 이어붙이기

In [63]:
np.concatenate([arr, arr])

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

#### - numpy 사용해서 열로 이어붙이기

In [64]:
np.concatenate([arr, arr], axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [65]:
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])

In [66]:
s1

a    0
b    1
dtype: int64

In [67]:
s2

c    2
d    3
e    4
dtype: int64

In [68]:
s3

f    5
g    6
dtype: int64

#### - pandas 사용해서 행으로 이어붙이기

In [70]:
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

#### - pandas 사용해서 열로 이어붙이기

In [71]:
pd.concat([s1, s2, s3], axis=1)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [72]:
s4 = pd.concat([s1, s3])
s4

a    0
b    1
f    5
g    6
dtype: int64

In [73]:
pd.concat([s1, s4], axis=1)

,0,1
a,0.0,0
b,1.0,1
f,NaN,5
g,NaN,6


In [75]:
pd.concat([s1, s4], axis=1, join='inner')

,0,1
a,0,0
b,1,1


#### - 합쳐지기 전 어떤 데이터에서 온 건지 보기 위해 keys 설정 해줄 수 있음

In [78]:
result = pd.concat([s1, s2, s3], keys=['one', 'two', 'three'])
result

one    a    0
       b    1
two    c    2
       d    3
       e    4
three  f    5
       g    6
dtype: int64

In [79]:
result.unstack()

,a,b,c,d,e,f,g
one,0.0,1.0,NaN,NaN,NaN,NaN,NaN
two,NaN,NaN,2.0,3.0,4.0,NaN,NaN
three,NaN,NaN,NaN,NaN,NaN,5.0,6.0


In [81]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'],
                   columns=['one', 'two'])
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=['a', 'c'],
                   columns=['three', 'four'])

In [82]:
df1

,one,two
a,0,1
b,2,3
c,4,5


In [83]:
df2

,three,four
a,5,6
c,7,8


In [85]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'])

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [86]:
df1 = pd.DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])
df2 = pd.DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])

In [87]:
df1

,a,b,c,d
0,1.124146,-3.402897,-0.469088,0.751337
1,0.785025,-0.569343,0.068882,-0.511022
2,-1.022241,-0.264181,-1.130542,-0.186919


In [88]:
df2

,b,d,a
0,-0.235512,0.220646,-1.373889
1,1.081472,0.065216,0.494795


#### - ignore_index=True: 무의미하게 생긴 인덱스 삭제함

In [89]:
pd.concat([df1, df2])

,a,b,c,d
0,1.124146,-3.402897,-0.469088,0.751337
1,0.785025,-0.569343,0.068882,-0.511022
2,-1.022241,-0.264181,-1.130542,-0.186919
0,-1.373889,-0.235512,NaN,0.220646
1,0.494795,1.081472,NaN,0.065216


In [91]:
pd.concat([df1, df2], ignore_index=True)

,a,b,c,d
0,1.124146,-3.402897,-0.469088,0.751337
1,0.785025,-0.569343,0.068882,-0.511022
2,-1.022241,-0.264181,-1.130542,-0.186919
3,-1.373889,-0.235512,NaN,0.220646
4,0.494795,1.081472,NaN,0.065216
